In [ ]:
import pandas as pd
import numpy as np
import ast
import imblearn
from imblearn.ensemble import BalancedRandomForestClassifier
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score, precision_recall_curve, auc
# Import necessary libraries
from google.colab import drive
from ast import literal_eval

In [ ]:
# Mount Google Drive
#drive.mount('/content/drive')

# Load the CSV file into a DataFrame
file_path = '/content/drive/MyDrive/CSM_ML&DL_Project/Code/Data/densenet_test_embeddings.csv'
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,path_to_image,path_to_dcm,age,sex,race,insurance_type,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,embeddings
0,train/patient47347/study3/view1_frontal.jpg,train/patient47347/study3/view1_frontal.dcm,78.0,1,0,1,0,0,1,0,...,1,0,0,1,0,1,0,1,1,"[0.0029132624622434378, 0.1020001769065857, 0...."
1,train/patient37527/study12/view1_frontal.jpg,train/patient37527/study12/view1_frontal.dcm,63.0,0,1,2,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"[0.0014348188415169716, 0.0543656125664711, 0...."
2,train/patient41208/study9/view1_frontal.jpg,train/patient41208/study9/view1_frontal.dcm,70.0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,"[0.001982336398214102, 0.040021587163209915, 0..."
3,train/patient39357/study1/view1_frontal.jpg,train/patient39357/study1/view1_frontal.dcm,79.0,1,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,"[0.001741771469824016, 0.0560498870909214, 0.1..."
4,train/patient31982/study4/view1_frontal.jpg,train/patient31982/study4/view1_frontal.dcm,67.0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,"[9.678312198957428e-05, 0.12247737497091293, 0..."


In [ ]:
df.columns

Index(['path_to_image', 'path_to_dcm', 'age', 'sex', 'race', 'insurance_type',
       'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices', 'embeddings'],
      dtype='object')

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/CSM_ML&DL_Project/Code/Data/densenet_test_embeddings.csv')
valid = pd.read_csv('/content/drive/MyDrive/CSM_ML&DL_Project/Code/Data/densenet_valid_embeddings.csv')

test['embeddings'] = test['embeddings'].apply(ast.literal_eval)
valid['embeddings'] = valid['embeddings'].apply(ast.literal_eval)
test = test.drop(columns=['path_to_image', 'path_to_dcm'])
valid = valid.drop(columns=['path_to_image', 'path_to_dcm'])

In [ ]:
initial_size = test.shape[0]

# The previous logic with transforming the list to string and filtering on the length of said string is not necessarily stable and misleading.
# Let's implement a more explicit test for what we actually care about:

test = test[test['embeddings'].apply(type) == list]

final_size = test.shape[0]

print(f'Number of test removed rows = {initial_size - final_size}')

initial_size = valid.shape[0]

valid = valid[valid['embeddings'].apply(type) == list]

final_size = valid.shape[0]

print(f'Number of train removed rows = {initial_size - final_size}')

Number of test removed rows = 51
Number of train removed rows = 3


In [ ]:
a = 70
test['age'] = (test['age'] >= a).astype(int)
valid['age'] = (valid['age'] >= a).astype(int)

In [ ]:
test_embeddings = pd.DataFrame(test['embeddings'].tolist())
valid_embeddings = pd.DataFrame(valid['embeddings'].tolist())


# Diseases to predict
# diseases = ['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
#             'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture']
diseases = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']

# Labels for train and test
y_test = test[diseases]
y_valid = valid[diseases]

# Create x_train and x_test
x_valid = pd.concat([valid.reset_index(), valid_embeddings], axis=1)
x_test =  pd.concat([test.reset_index(), test_embeddings], axis=1)

x_valid.drop(columns=["embeddings"] + diseases, inplace=True)
x_test.drop(columns=["embeddings"] + diseases, inplace=True)



# Labels for train and test
y_test = test[diseases]
y_valid = valid[diseases]
y_no_finding = valid["No Finding"]
y_sex = valid['sex']
y_race = valid['race']
y_insurance = valid['insurance_type']
y_age = valid['age']

#### Performing PCA to find optimal number of components with 95% variance threshold

In [ ]:
# Step 1: Standardize the embeddings_list to have mean 0 and variance 1
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(np.stack(test_embeddings.values))
X_valid_scaled = scaler.transform(np.stack(valid_embeddings.values))

# Step 2: Set target variance threshold (e.g., 95%)
variance_threshold = 0.95

# Step 3: Fit PCA to determine the optimal number of components based on variance threshold
pca_full = PCA()
pca_full.fit(X_test_scaled)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Step 4: Find the number of components that meets the variance threshold
optimal_components = np.argmax(cumulative_variance >= variance_threshold) + 1
print(f"Optimal number of components to retain {variance_threshold*100}% variance: {optimal_components}")

#95% variance means that the selected principal components (reduced dimensions) retain 95% of the total variability present in the original high-dimensional data.

Optimal number of components to retain 95.0% variance: 302


### Dimensionality reduction of Embeddings with PCA & Training BalancedRandomClassifier

In [ ]:
pca = PCA(n_components=302, random_state=42)
X_test_reduced = pca.fit_transform(X_test_scaled)
X_valid_reduced = pca.transform(X_valid_scaled)

# Prepare y_train and y_test as before
y_test = test[['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']].values

y_valid = valid[['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']].values

# Step 6: Initialize and train the BalancedRandomForestClassifier on the reduced data
classifiers = []
predictions = []
results = []

for i, label in enumerate(['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']):

    # Train a separate BalancedRandomForestClassifier for each label (one-vs-all)

    # Create a dynamic variable name for each classifier
    clf_name = f"clf_{i}"
    globals()[clf_name] = BalancedRandomForestClassifier(random_state=42, n_jobs=-1)
    globals()[clf_name].fit(X_test_reduced, y_test[:, i])
    classifiers.append(globals()[clf_name])

    # Predict on valid data
    y_pred = globals()[clf_name].predict(X_valid_reduced)
    y_prob = globals()[clf_name].predict_proba(X_valid_reduced)[:, 1]  # probability for the positive class

    # Calculate metrics
    accuracy = accuracy_score(y_valid[:, i], y_pred)
    precision = precision_score(y_valid[:, i], y_pred, zero_division=1)
    recall = recall_score(y_valid[:, i], y_pred, zero_division=1)
    f1 = f1_score(y_valid[:, i], y_pred, zero_division=1)
    auc = roc_auc_score(y_valid[:, i], y_prob)
    #auprc = sklearn.metrics.auc(recall, precision)
    # Confusion Matrix for calculating rates
    tn, fp, fn, tp = confusion_matrix(y_valid[:, i], y_pred).ravel()
    tp_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
    fn_rate = fn / (tp + fn) if (tp + fn) > 0 else 0
    tn_rate = tn / (tn + fp) if (tn + fp) > 0 else 0
    fp_rate = fp / (tn + fp) if (tn + fp) > 0 else 0

    # Store results
    results.append({
        'label': label,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        # 'AUPRC': auprc,
        'tp_rate': tp_rate,
        'fn_rate': fn_rate,
        'tn_rate': tn_rate,
        'fp_rate': fp_rate
    })

    # Store predictions along with demographic information
    predictions.append({
        'label': label,
        'sex': valid['sex'].values,
        'race': valid['race'].values,
        'age': valid['age'].values,
        'insurance_type': valid['insurance_type'].values,
        'y_pred': y_pred,
        'y_actual': y_valid[:, i]
    })

# Convert results list to a DataFrame for easy viewing
results_df = pd.DataFrame(results)
print("Evaluation Metrics for Each Label after PCA Dimensionality Reduction:")
print(results_df)

# Convert predictions list to a DataFrame for easy viewing
predictions_df = pd.concat([pd.DataFrame(pred) for pred in predictions], ignore_index=True)
print("Predictions and Demographics:")
print(predictions_df)

/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.10/dist-packages/imblearn/ensemble/_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
/usr/local/lib/python3.

Evaluation Metrics for Each Label after PCA Dimensionality Reduction:
              label  accuracy  precision    recall  f1_score       auc  \
0      Cardiomegaly  0.713010   0.230664  0.675351  0.343878  0.759293   
1      Lung Opacity  0.620174   0.617100  0.658440  0.637100  0.668916   
2             Edema  0.687570   0.410504  0.718574  0.522510  0.766391   
3       Atelectasis  0.570632   0.189576  0.523272  0.278320  0.576290   
4      Pneumothorax  0.646507   0.166371  0.614880  0.261883  0.684542   
5  Pleural Effusion  0.695380   0.602818  0.747271  0.667317  0.761764   

    tp_rate   fn_rate   tn_rate   fp_rate  
0  0.675351  0.324649  0.717730  0.282270  
1  0.658440  0.341560  0.580922  0.419078  
2  0.718574  0.281426  0.677892  0.322108  
3  0.523272  0.476728  0.579533  0.420467  
4  0.614880  0.385120  0.650099  0.349901  
5  0.747271  0.252729  0.659494  0.340506  
Predictions and Demographics:
                  label  sex  race  age  insurance_type  y_pred  y_actual

In [ ]:
predictions_df

,label,sex,race,age,insurance_type,y_pred,y_actual
0,Cardiomegaly,1,0,0,1,1,0
1,Cardiomegaly,0,0,1,1,0,0
2,Cardiomegaly,1,0,0,2,1,0
3,Cardiomegaly,0,0,0,1,0,0
4,Cardiomegaly,1,2,1,1,1,0
...,...,...,...,...,...,...,...
26881,Pleural Effusion,0,2,0,1,0,0
26882,Pleural Effusion,0,0,0,2,1,1
26883,Pleural Effusion,0,0,1,1,1,0
26884,Pleural Effusion,1,1,1,1,1,0


In [ ]:
predictions1 = y_pred
targets1 = y_valid

In [ ]:
diseases = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']

In [ ]:
results_df

,label,accuracy,precision,recall,f1_score,auc,tp_rate,fn_rate,tn_rate,fp_rate
0,Cardiomegaly,0.713010,0.230664,0.675351,0.343878,0.759293,0.675351,0.324649,0.717730,0.282270
1,Lung Opacity,0.620174,0.617100,0.658440,0.637100,0.668916,0.658440,0.341560,0.580922,0.419078
2,Edema,0.687570,0.410504,0.718574,0.522510,0.766391,0.718574,0.281426,0.677892,0.322108
3,Atelectasis,0.570632,0.189576,0.523272,0.278320,0.576290,0.523272,0.476728,0.579533,0.420467
4,Pneumothorax,0.646507,0.166371,0.614880,0.261883,0.684542,0.614880,0.385120,0.650099,0.349901
5,Pleural Effusion,0.695380,0.602818,0.747271,0.667317,0.761764,0.747271,0.252729,0.659494,0.340506


In [ ]:
import sklearn
metrics = {}
for i, label in enumerate(['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']):
    # Use the already trained classifier to predict on validation data
    y_valid_pred = globals()[f"clf_{i}"].predict(X_valid_reduced)
    y_valid_prob = globals()[f"clf_{i}"].predict_proba(X_valid_reduced)[:, 1]  # probability for the positive class

    # Calculate metrics
    accuracy = accuracy_score(y_valid[:, i], y_valid_pred)
    precision_values, recall_values, _ = precision_recall_curve(y_valid[:, i], y_valid_prob)
    f1 = f1_score(y_valid[:, i], y_valid_pred, zero_division=1)
    auc_roc = roc_auc_score(y_valid[:, i], y_valid_prob)
    auprc_value = sklearn.metrics.auc(recall_values, precision_values)

    # Confusion Matrix for calculating rates
    tn, fp, fn, tp = confusion_matrix(y_valid[:, i], y_valid_pred).ravel()
    tp_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
    fn_rate = fn / (tp + fn) if (tp + fn) > 0 else 0
    tn_rate = tn / (tn + fp) if (tn + fp) > 0 else 0
    fp_rate = fp / (tn + fp) if (tn + fp) > 0 else 0

    # Store metrics in dictionary
    metrics[label] = {
        'Accuracy': accuracy,
        'AUC': auc_roc,
        'AUPRC': auprc_value,
        'F1 Score': f1,
        'TP Rate': tp_rate,
        'FN Rate': fn_rate,
        'TN Rate': tn_rate,
        'FP Rate': fp_rate
    }

print("Validation Metrics for Each Label:")
print(metrics)

Validation Metrics for Each Label:
{'Cardiomegaly': {'Accuracy': 0.7130104887301941, 'AUC': 0.7592925680592727, 'AUPRC': 0.29584618303055876, 'F1 Score': 0.3438775510204082, 'TP Rate': 0.6753507014028056, 'FN Rate': 0.3246492985971944, 'TN Rate': 0.7177297840281266, 'FP Rate': 0.2822702159718734}, 'Lung Opacity': {'Accuracy': 0.6201740682883286, 'AUC': 0.6689158737508538, 'AUPRC': 0.633770725888823, 'F1 Score': 0.6371002132196162, 'TP Rate': 0.6584398413397973, 'FN Rate': 0.34156015866020273, 'TN Rate': 0.5809222423146474, 'FP Rate': 0.4190777576853526}, 'Edema': {'Accuracy': 0.6875697388975676, 'AUC': 0.7663911009534693, 'AUPRC': 0.47240189205232896, 'F1 Score': 0.5225102319236017, 'TP Rate': 0.7185741088180112, 'FN Rate': 0.28142589118198874, 'TN Rate': 0.677891654465593, 'FP Rate': 0.32210834553440704}, 'Atelectasis': {'Accuracy': 0.5706315554563713, 'AUC': 0.5762899966646076, 'AUPRC': 0.19369488098613144, 'F1 Score': 0.27831957989497375, 'TP Rate': 0.5232722143864598, 'FN Rate': 0.

In [ ]:
metrics_old = metrics

In [ ]:
metrics = {}
diseases = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion']

for subgroup, filter_condition in {
    'female': (predictions_df['sex'] == 1),
    'male': (predictions_df['sex'] == 0),
    'white': (predictions_df['race'] == 0),
    'black': (predictions_df['race'] == 2),
    'asian': (predictions_df['race'] == 1),
    'medicaid': (predictions_df['insurance_type'] == 0),
    'medicare': (predictions_df['insurance_type'] == 1),
    'private': (predictions_df['insurance_type'] == 2),
    'age_above_70' : (predictions_df['age'] ==1 ),
    'age_below_70' : (predictions_df['age'] ==0 )

}.items():
    subgroup_metrics = {}
    for idx, disease in enumerate(diseases):
        disease_pred = predictions_df.loc[filter_condition & (predictions_df['label'] == disease), 'y_pred']
        disease_true = predictions_df.loc[filter_condition & (predictions_df['label'] == disease), 'y_actual']

        auc_roc = roc_auc_score(disease_true, disease_pred)
        f1 = f1_score(disease_true, (disease_pred > 0.5).astype(int))
        accuracy = accuracy_score(disease_true, (disease_pred > 0.5).astype(int))
        tn, fp, fn, tp = confusion_matrix(disease_true, (disease_pred > 0.5).astype(int)).ravel()
        tp_rate = tp / (tp + fn) if (tp + fn) > 0 else 0
        tn_rate = tn / (tn + fp) if (tn + fp) > 0 else 0
        fn_rate = fn / (fn + tp) if (fn + tp) > 0 else 0
        fp_rate = fp / (tn + fp) if (tn + fp) > 0 else 0
        precision, recall, _ = precision_recall_curve(disease_true, disease_pred)
        auprc = sklearn.metrics.auc(recall, precision)

        subgroup_metrics[disease] = {
            'Accuracy': accuracy,
            'AUC': auc_roc,
            'AUPRC': auprc,
            'F1 Score': f1,
            'TP Rate': tp_rate,
            'FN Rate': fn_rate,
            'TN Rate': tn_rate,
            'FP Rate': fp_rate
        }
    metrics[subgroup] = subgroup_metrics

print("Metrics by Subgroup:")
print(metrics)


Metrics by Subgroup:
{'female': {'Cardiomegaly': {'Accuracy': 0.7150451887293993, 'AUC': 0.6722701603002562, 'AUPRC': 0.43743389405313154, 'F1 Score': 0.3197969543147208, 'TP Rate': 0.6176470588235294, 'FN Rate': 0.38235294117647056, 'TN Rate': 0.7268932617769827, 'FP Rate': 0.2731067382230173}, 'Lung Opacity': {'Accuracy': 0.6220095693779905, 'AUC': 0.6218201223786501, 'AUPRC': 0.7220932119370779, 'F1 Score': 0.6317969963749352, 'TP Rate': 0.642781875658588, 'FN Rate': 0.35721812434141204, 'TN Rate': 0.6008583690987125, 'FP Rate': 0.39914163090128757}, 'Edema': {'Accuracy': 0.7091972355130249, 'AUC': 0.7037993878296033, 'AUPRC': 0.5984507294777518, 'F1 Score': 0.5312767780634104, 'TP Rate': 0.6935123042505593, 'FN Rate': 0.30648769574944074, 'TN Rate': 0.7140864714086471, 'FP Rate': 0.2859135285913529}, 'Atelectasis': {'Accuracy': 0.5773524720893142, 'AUC': 0.5404041265240003, 'AUPRC': 0.37513394184296345, 'F1 Score': 0.2659279778393352, 'TP Rate': 0.4864864864864865, 'FN Rate': 0.513

### ML performance : Race-wise per Disease

In [ ]:
# Create an empty list to store results
disease_data = []

# Extract disease names (assuming all demographics have the same diseases)
diseases = metrics['female'].keys()

# Iterate over diseases to collect relevant metrics for race
for disease in diseases:
    # Metrics for AUC and AUPRC for each race
    auc_values = {race: metrics[race][disease]['AUC'] for race in ['asian', 'white', 'black']}
    auprc = metrics['female'][disease]['AUPRC']  # Assuming AUPRC is the same across all races for simplicity

    # Calculate max delta AUC and equality of odds between races
    auc_asian = auc_values['asian']
    auc_white = auc_values['white']
    auc_black = auc_values['black']
    max_delta_auc = max(abs(auc_white - auc_black), abs(auc_white - auc_asian), abs(auc_black - auc_asian))

    tp_rates = {race: metrics[race][disease]['TP Rate'] for race in ['asian', 'white', 'black']}
    fp_rates = {race: metrics[race][disease]['FP Rate'] for race in ['asian', 'white', 'black']}
    max_eq_odds = max(
        0.5 * (abs(tp_rates['asian'] - tp_rates['white']) + abs(fp_rates['asian'] - fp_rates['white'])),
        0.5 * (abs(tp_rates['asian'] - tp_rates['black']) + abs(fp_rates['asian'] - fp_rates['black'])),
        0.5 * (abs(tp_rates['white'] - tp_rates['black']) + abs(fp_rates['white'] - fp_rates['black']))
    )

    # Append the metrics to the list
    disease_data.append({
        'Disease': disease,
        'AUPRC': auprc,
        'AUC': (auc_asian + auc_white + auc_black) / 3,  # Mean AUC for all races
        'AUC_White': auc_white,
        'AUC_Black': auc_black,
        'AUC_Asian': auc_asian,
        'Max Delta AUC': max_delta_auc,
        'Max EqOdds': max_eq_odds
    })

# Convert the list of dictionaries to a DataFrame
df_race = pd.DataFrame(disease_data)

# Styling the DataFrame
styled_df_race = df_race.style.format({
    'AUC': "{:.3f}",
    'AUPRC': "{:.3f}",
    'AUC_White': "{:.3f}",
    'AUC_Black': "{:.3f}",
    'AUC_Asian': "{:.3f}",
    'Max Delta AUC': "{:.3f}",
    'Max EqOdds': "{:.3f}"
}).background_gradient(cmap='viridis', subset=['AUPRC', 'AUC', 'AUC_White', 'AUC_Black', 'AUC_Asian', 'Max Delta AUC', 'Max EqOdds'])

# Display the styled DataFrame
styled_df_race

,Disease,AUPRC,AUC,AUC_White,AUC_Black,AUC_Asian,Max Delta AUC,Max EqOdds
0,Cardiomegaly,0.437,0.692,0.696,0.717,0.664,0.053,0.087
1,Lung Opacity,0.722,0.617,0.619,0.606,0.625,0.019,0.047
2,Edema,0.598,0.696,0.700,0.707,0.681,0.026,0.039
3,Atelectasis,0.375,0.543,0.552,0.530,0.547,0.022,0.088
4,Pneumothorax,0.428,0.639,0.628,0.646,0.644,0.018,0.087
5,Pleural Effusion,0.724,0.699,0.702,0.676,0.719,0.044,0.044


## ML performance : Age-wise per Disease

In [ ]:
# Create an empty list to store results
disease_data = []

# Extract disease names (assuming all demographics have the same diseases)
diseases = metrics['female'].keys()

# Iterate over diseases to collect relevant metrics for age groups
for disease in diseases:
    # Metrics for AUC and AUPRC for each age group
    auc_old = metrics['age_above_70'][disease]['AUC']
    auc_young = metrics['age_below_70'][disease]['AUC']
    auprc = metrics['female'][disease]['AUPRC']  # Assuming AUPRC is the same across all age groups for simplicity

    # Calculate delta AUC and equality of odds between age groups
    delta_auc_age = abs(auc_old - auc_young)

    tp_rate_old = metrics['age_above_70'][disease]['TP Rate']
    tp_rate_young = metrics['age_below_70'][disease]['TP Rate']
    fp_rate_old = metrics['age_above_70'][disease]['FP Rate']
    fp_rate_young = metrics['age_below_70'][disease]['FP Rate']
    eq_odds_age = 0.5 * (abs(tp_rate_old - tp_rate_young) + abs(fp_rate_old - fp_rate_young))

    # Append the metrics to the list
    disease_data.append({
        'Disease': disease,
        'AUPRC': auprc * 100,  # Scaling AUPRC to match the provided output format
        'AUC': (auc_old + auc_young) / 2,  # Mean AUC for both age groups
        'AUC_old': auc_old,
        'AUC_young': auc_young,
        'Delta AUC': delta_auc_age,
        'EqOdds': eq_odds_age
    })

# Convert the list of dictionaries to a DataFrame
df_age = pd.DataFrame(disease_data)

# Styling the DataFrame
styled_df_age = df_age.style.format({
    'AUC': "{:.3f}",
    'AUPRC': "{:.3f}",
    'AUC_old': "{:.3f}",
    'AUC_young': "{:.3f}",
    'Delta AUC': "{:.3f}",
    'EqOdds': "{:.3f}"
}).background_gradient(cmap='viridis', subset=['AUPRC', 'AUC', 'AUC_old', 'AUC_young', 'Delta AUC', 'EqOdds'])

# Display the styled DataFrame
styled_df_age


,Disease,AUPRC,AUC,AUC_old,AUC_young,Delta AUC,EqOdds
0,Cardiomegaly,43.743,0.688,0.663,0.714,0.051,0.069
1,Lung Opacity,72.209,0.608,0.587,0.630,0.043,0.132
2,Edema,59.845,0.693,0.688,0.699,0.011,0.096
3,Atelectasis,37.513,0.548,0.534,0.562,0.028,0.028
4,Pneumothorax,42.806,0.634,0.639,0.630,0.009,0.009
5,Pleural Effusion,72.402,0.698,0.692,0.704,0.012,0.102


### ML performance : Insurance-wise per Disease

In [ ]:
# Create an empty list to store results
disease_data = []

# Extract disease names (assuming all demographics have the same diseases)
diseases = metrics['female'].keys()

# Iterate over diseases to collect relevant metrics for insurance types
for disease in diseases:
    # Metrics for AUC and AUPRC for each insurance type
    auc_medicaid = metrics['medicaid'][disease]['AUC']
    auc_medicare = metrics['medicare'][disease]['AUC']
    auc_private = metrics['private'][disease]['AUC']
    auprc = metrics['female'][disease]['AUPRC']  # Assuming AUPRC is the same across all insurance types for simplicity

    # Calculate max delta AUC and equality of odds between insurance types
    max_delta_auc = max(abs(auc_medicaid - auc_medicare), abs(auc_medicaid - auc_private), abs(auc_medicare - auc_private))

    tp_rate_medicaid = metrics['medicaid'][disease]['TP Rate']
    tp_rate_medicare = metrics['medicare'][disease]['TP Rate']
    tp_rate_private = metrics['private'][disease]['TP Rate']

    fp_rate_medicaid = metrics['medicaid'][disease]['FP Rate']
    fp_rate_medicare = metrics['medicare'][disease]['FP Rate']
    fp_rate_private = metrics['private'][disease]['FP Rate']

    max_eq_odds = max(
        0.5 * (abs(tp_rate_medicaid - tp_rate_medicare) + abs(fp_rate_medicaid - fp_rate_medicare)),
        0.5 * (abs(tp_rate_medicaid - tp_rate_private) + abs(fp_rate_medicaid - fp_rate_private)),
        0.5 * (abs(tp_rate_medicare - tp_rate_private) + abs(fp_rate_medicare - fp_rate_private))
    )

    # Append the metrics to the list
    disease_data.append({
        'Disease': disease,
        'AUPRC': auprc * 100,  # Scaling AUPRC to match the provided output format
        'AUC': (auc_medicaid + auc_medicare + auc_private) / 3,  # Mean AUC for all insurance types
        'AUC_Medicaid': auc_medicaid,
        'AUC_Medicare': auc_medicare,
        'AUC_Private': auc_private,
        'Max Delta AUC': max_delta_auc,
        'Max EqOdds': max_eq_odds
    })

# Convert the list of dictionaries to a DataFrame
df_insurance = pd.DataFrame(disease_data)

# Styling the DataFrame
styled_df_insurance = df_insurance.style.format({
    'AUC': "{:.3f}",
    'AUPRC': "{:.3f}",
    'AUC_Medicaid': "{:.3f}",
    'AUC_Medicare': "{:.3f}",
    'AUC_Private': "{:.3f}",
    'Max Delta AUC': "{:.3f}",
    'Max EqOdds': "{:.3f}"
}).background_gradient(cmap='viridis', subset=['AUPRC', 'AUC', 'AUC_Medicaid', 'AUC_Medicare', 'AUC_Private', 'Max Delta AUC', 'Max EqOdds'])

# Display the styled DataFrame
styled_df_insurance


,Disease,AUPRC,AUC,AUC_Medicaid,AUC_Medicare,AUC_Private,Max Delta AUC,Max EqOdds
0,Cardiomegaly,43.743,0.683,0.679,0.699,0.671,0.027,0.106
1,Lung Opacity,72.209,0.626,0.635,0.605,0.638,0.033,0.109
2,Edema,59.845,0.696,0.692,0.694,0.701,0.009,0.086
3,Atelectasis,37.513,0.579,0.624,0.528,0.584,0.096,0.096
4,Pneumothorax,42.806,0.614,0.564,0.634,0.643,0.079,0.085
5,Pleural Effusion,72.402,0.697,0.685,0.701,0.705,0.020,0.075


### Complete ML performance across sex, age, race, insurance subgroups

In [ ]:
# Create an empty list to store results
disease_data = []

# Extract disease names (assuming all demographics have the same diseases)
diseases = metrics['female'].keys()

# Iterate over diseases to collect relevant metrics for combined analysis
for i, disease in enumerate(diseases):
    # Metrics for AUC and AUPRC
    auprc = metrics['female'][disease]['AUPRC'] * 100  # Scaling AUPRC to match the provided output format
    auc = (metrics['medicaid'][disease]['AUC'] + metrics['medicare'][disease]['AUC'] + metrics['private'][disease]['AUC']) / 3  # Mean AUC for all insurance types

    # Collect delta AUC and equality of odds metrics from different groups
    delta_auc_sex = df_sex['Delta AUC'][i]
    eq_odds_sex = df_sex['EqOdds'][i]
    delta_auc_race = df_race['Max Delta AUC'][i]
    eq_odds_race = df_race['Max EqOdds'][i]
    delta_auc_age = df_age['Delta AUC'][i]
    eq_odds_age = df_age['EqOdds'][i]
    delta_auc_health = df_insurance['Max Delta AUC'][i]
    eq_odds_health = df_insurance['Max EqOdds'][i]

    # Append the metrics to the list
    disease_data.append({
        'Disease': disease,
        'AUPRC': auprc,
        'AUC': auc,
        'Delta AUC sex': delta_auc_sex,
        'EqOdds sex': eq_odds_sex,
        'Delta AUC race': delta_auc_race,
        'EqOdds race': eq_odds_race,
        'Delta AUC age': delta_auc_age,
        'EqOdds age': eq_odds_age,
        'Delta AUC health': delta_auc_health,
        'EqOdds health': eq_odds_health
    })

# Convert the list of dictionaries to a DataFrame
df_combined = pd.DataFrame(disease_data)

# Styling the DataFrame
styled_df_combined = df_combined.style.format({
    'AUC': "{:.3f}",
    'AUPRC': "{:.3f}",
    'Delta AUC sex': "{:.3f}",
    'EqOdds sex': "{:.3f}",
    'Delta AUC race': "{:.3f}",
    'EqOdds race': "{:.3f}",
    'Delta AUC age': "{:.3f}",
    'EqOdds age': "{:.3f}",
    'Delta AUC health': "{:.3f}",
    'EqOdds health': "{:.3f}"
}).background_gradient(cmap='viridis', subset=['AUC', 'AUPRC', 'Delta AUC sex', 'EqOdds sex', 'Delta AUC race', 'EqOdds race', 'Delta AUC age', 'EqOdds age', 'Delta AUC health', 'EqOdds health'])

# Display the styled DataFrame
styled_df_combined


,Disease,AUPRC,AUC,Delta AUC sex,EqOdds sex,Delta AUC race,EqOdds race,Delta AUC age,EqOdds age,Delta AUC health,EqOdds health
0,Cardiomegaly,43.743,0.683,0.041,0.057,0.053,0.087,0.051,0.069,0.027,0.106
1,Lung Opacity,72.209,0.626,0.004,0.031,0.019,0.047,0.043,0.132,0.033,0.109
2,Edema,59.845,0.696,0.010,0.053,0.026,0.039,0.011,0.096,0.009,0.086
3,Atelectasis,37.513,0.579,0.019,0.044,0.022,0.088,0.028,0.028,0.096,0.096
4,Pneumothorax,42.806,0.614,0.025,0.025,0.018,0.087,0.009,0.009,0.079,0.085
5,Pleural Effusion,72.402,0.697,0.023,0.023,0.044,0.044,0.012,0.102,0.020,0.075
